In [2]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.python.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)

# Normalize
x_train = x_train/255.
x_test  = x_test/255.

In [ ]:
def make_masking_noised_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size)
    return data_x * masking

x_train_masked = make_masking_noised_data(x_train)
x_test_masked  = make_masking_noised_data(x_test)

In [ ]:
def make_gaussian_noised_data(data_x, scale=0.8):
    gaussed_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
    gaussed_x = np.clip(gaussed_x, 0, 1)
    return gaussed_x

x_train_gauss = make_gaussian_noised_data(x_train)
x_test_gauss  = make_gaussian_noised_data(x_test)

In [ ]:
from IPython.display import display_png
